In [143]:
library(umap)

In [173]:
library(survival)
library(reshape2)
library(tibble)
library(xtable)

## Plotting
library(ggplot2)
library(cowplot)
library(pROC)

library(RColorBrewer)
library(pheatmap)
# library(Rtsne)
# library(dendextend)
# library(rgl)
# library(VennDiagram)
# library(UpSetR)
# library(repr)
# options(repr.plot.width=8, repr.plot.height=4)

theme_set(theme_bw())

## Custom
source("../functions.R")

In [145]:
# FUNCTIONS ---------------------------------------------------------------
#' @param y_true numeric vector of true labels with 1 as positive
calc_recall <- function(y_true, y_pred) {
  sum(y_pred[y_true == 1] == 1) / sum(y_true == 1)
}
#' @param y_true numeric vector of true labels with 1 as positive
calc_specificity <- function(y_true, y_pred) {
  sum(y_pred[y_true == 0] == 0) / sum(y_true == 0)
} 

#' Sorts sample IDs so that they are paired
sort_sid <- function(x) {
  pid <- sapply(x, substring, 1, 4)
  time <- sapply(x, substring, 6, 7)
  time_pid <- mapply(paste0, time, pid)
  return(x[order(time_pid)])
}

#' Boolean function checking that dataframe provided has matching pair names
#' @param X dataframe or character
#' @return logical indicating if it is paired or not
is_paired <- function(x) {
  #' @param character of sample ids
  #' @return logical indicating if it is paired or not
  .is_paired <- function(sid) {
    n <- length(sid)
    pid <- substring(sid, 1, 4)
    all(pid[1:(n / 2)] == pid[(n / 2 + 1):n])
  }
  
  if (is.data.frame(x) | is.matrix(x)) {
    return(.is_paired(colnames(x)))
  } else if (is.character(x)) {
    return(.is_paired(x))
  } else{
    stop("x is not a dataframe, matrix or character.")
  }
}

In [146]:
 #' @param response_df dataframe with samples x features
#' @param normal_df dataframe with samples x features
# D0 centroid used to define D0-Normal vector
compute_features <- function(
  response_df, normal_df,
  sid_train, sid_remission
) {
  print(rownames(response_df))
  # Split response df into D0 and D8 df
  n <- nrow(response_df)/2
  d0_df <- response_df[1:n, , drop = F]
  d8_df <- response_df[-(1:n), , drop = F]
  
  if (!is_paired(t(response_df)))
    stop("Patient IDs are not paired..")
  
  # Calculate centroids
  # Only use remission patients in training set to calculate centroid
  sid_leuk <- Reduce(
    intersect,
    list(rownames(d0_df), sid_train, sid_remission)
  )
  print(rownames(d0_df))
  print(sprintf("NO. OF SAMPLES IN CENTROID: %d", length(sid_leuk)))
  leuk_centroid <- apply(d0_df[sid_leuk, , drop = F], 2, median)
  normal_centroid <- apply(normal_df, 2, median)
  
  # Calculate leuk-normal unit vector
  leuk_normal <- normal_centroid - leuk_centroid
  unit_leuk_normal <- leuk_normal/calcL2Norm(leuk_normal)
  
  # Assume that patients from top rows match correspondingly with bottom rows
  # Calculate vector by: D8-D0
  d0_d8_hstack <- d8_df - d0_df
  # Multiplication of erm_factor is propagated through every column
  ### ERM1 ###
  erm1 <- colSums(t(d0_d8_hstack) * unit_leuk_normal)
  # Vertical stack of individual D0-Normal vectors
  d0_normal_vstack <- normal_centroid - t(d0_df)
  ### D0-Normal projection ###
  d0_normal_proj <- colSums(d0_normal_vstack * unit_leuk_normal)
  ### ERM1 Ratio ###
  ## ERM1 / projection of D0-N on L-N
  erm1_ratio1 <- erm1/d0_normal_proj
  
  d8_normal_vstack <- normal_centroid - t(d8_df)
  ### D8-Normal projection ###
  d8_normal_proj <- colSums(d8_normal_vstack * unit_leuk_normal)
  
  stopifnot(identical(names(erm1), names(erm1_ratio1)))
  
  # Calculate vstack of unit D0-Normal vectors
  l2norm_d0_normal <- apply(d0_normal_vstack, 2, calcL2Norm)
  unit_d0_normal_vstack <- sweep(d0_normal_vstack, 2, l2norm_d0_normal, "/")
  
  ### ERM2 ###
  ## Projection of D0-D8 on D0-N
  erm2 <- colSums(t(d0_d8_hstack) * unit_d0_normal_vstack)
  erm2_ratio <- erm2/l2norm_d0_normal
  
  stopifnot(identical(names(erm2), names(erm2_ratio)))
  
  ### ERM3 ###
  ## Along a chosen PC that represents timepoint
  PC <- 1
  # Be careful of direction of D0-N (may be negative)
  # If negative, a larger shift will lead to a smaller ERM3
  dir <- sign(median(normal_df[,PC]) - median(d0_df[,PC]))
  erm3 <- (d8_df[,PC] - d0_df[,PC]) * dir # direction is normalised
  # Divide by D0-Normal along PC
  erm3_ratio <- erm3/(median(normal_df[,PC]) - d0_df[,PC])
  
  stopifnot(identical(names(erm3), names(erm3_ratio)))
  
  ### l2norm ###
  l2norm_d0_d8 <- apply(d0_d8_hstack, 1, calcL2Norm)
  l2norm_d0 <- apply(d0_df, 1, calcL2Norm)
  l2norm_d8 <- apply(d8_df, 1, calcL2Norm)
  diff_l2norm <- l2norm_d8 - l2norm_d0
  
  ### Angle between D0-D8 and Leuk-Normal ###
  angle_d0d8_normal <- apply(
    d0_d8_hstack, 1, function(row_vec) calcAngleVectors(row_vec, leuk_normal)
  )
  
  ### Angle between D0-D8 and D0-Normal ###
  angle_d0d8_d0normal <- mapply(calcAngleVectors,
                                data.frame(t(d0_d8_hstack)),
                                data.frame(d0_normal_vstack))
  
  ### Angle between D0 and D8 ###
  angle_d0_d8 <- mapply(calcAngleVectors,
                        data.frame(t(d0_df)), data.frame(t(d8_df)))
  
  ### Angle between D0 and normal ###
  angle_d0_normal <- apply(
    d0_df, 1, function(row_vec) calcAngleVectors(row_vec, normal_centroid)
  )
  
  ### Angle between D8 and Normal ###
  angle_d8_normal <- apply(
    d8_df, 1, function(row_vec) calcAngleVectors(row_vec, normal_centroid)
  )
  
  ### Angle between N-D0 and N-D8 ###
  # Equivalent to angle between D0-N and D8-N
  angle_nd0_nd8 <- mapply(calcAngleVectors,
                          data.frame(d0_normal_vstack),
                          data.frame(d8_normal_vstack))
  
  ### Angle between N-centroid(D0) N-D8 ###
  # Equivalent to angle between centroid(D0)-N and D8-N
  angle_nl_nd8 <- sapply(data.frame(d8_normal_vstack),
                         function(x, y) calcAngleVectors(x, y),
                         leuk_normal)
  
  ### L2-norm between D8 and Normal ###
  l2norm_d8_normal <- apply(d8_normal_vstack, 2, calcL2Norm)
  
  ### L2-norm ratios
  l2norm_ratio1 <- l2norm_d0_d8/l2norm_d0_normal
  l2norm_ratio2 <- l2norm_d0_d8/l2norm_d8_normal
  l2norm_diff <- l2norm_d0_normal - l2norm_d8_normal
  l2norm_diff_ratio <- l2norm_diff/l2norm_d0_d8
  
  ### Ratios
  erm1_ratio2 <- erm1/abs(d8_normal_proj)
  erm1_ratio3 <- erm1/l2norm_d0_d8
  
  ### Concatenate all features ###
  features_df <- data.frame(
    erm1, erm1_ratio1, erm2, erm2_ratio, erm3, erm3_ratio,
    d0_normal_proj, d8_normal_proj, l2norm_d0_d8,
    diff_l2norm, angle_d0_d8, angle_nd0_nd8, angle_nl_nd8,
    angle_d0d8_normal, angle_d0d8_d0normal,
    angle_d0_normal, angle_d8_normal,
    l2norm_d0_normal, l2norm_d8_normal,
    l2norm_ratio1, l2norm_ratio2,
    l2norm_diff, l2norm_diff_ratio,
    erm1_ratio2, erm1_ratio3
  )
  
  rownames(features_df) <- substring(rownames(features_df), 1, 4)
  return(features_df)
}

## Import data

In [148]:
## Metadata
METADATA_RPATH <- "data/GSE67684/processed/metadata/full_metadata.tsv"
BATCH_RPATH <- "data/GSE67684/processed/metadata/metadata-batch.tsv"
LABEL_RPATH <- "data/GSE67684/processed/metadata/pid-mrd_label_tte_subtype.tsv"

metadata_df <- read.table(METADATA_RPATH, sep = "\t")
yeoh_batch <- read.table(BATCH_RPATH, sep = "\t", header = T, row.names = 1)
yeoh_label <- read.table(LABEL_RPATH, sep = "\t", header = T, row.names = 1)

## Subset of original data
# Removed outliers, patients with timepoints from different batches and batch 5
SUBSET_RPATH <- "data/GSE67684/processed/subset_yeoh.tsv"
raw_yeoh <- read.table(SUBSET_RPATH, sep = "\t")
# SCALE->REMOVE->FILTER->LOG
scaled_yeoh <- normaliseMeanScaling(raw_yeoh)
selected_yeoh <- removeProbesets(scaled_yeoh)
data <- log2_transform(filterProbesets(selected_yeoh, 0.7, metadata_df))

  P001_D0   P004_D0   P005_D0   P007_D0   P008_D0   P009_D0 
 5.796952  4.123342  3.981577  6.317643  4.841458 11.978124 
[1] "No. of ambiguous and AFFY probesets removed: 10503"
[1] D0 D0 D0 D0 D0 D0
Levels: D0 D8 N
           D0    D8     N
1053_at  TRUE  TRUE  TRUE
117_at  FALSE  TRUE  TRUE
121_at   TRUE  TRUE  TRUE
1294_at  TRUE  TRUE  TRUE
1316_at  TRUE  TRUE  TRUE
1320_at FALSE FALSE FALSE
[1] "No. of probesets removed = 6321"


### Commonly used globals

In [151]:
COL_LABEL <- c("darkolivegreen3", "tomato3")

Y <- metadata_df[colnames(data),]
Y_annot <- Y[,c("batch_info", "label")] # heatmap annot
# Y_all <- all_metadata[colnames(data_all),]

# List subtypes
subtypes9 <- levels(metadata_df$subtype)
subtypes7 <- setdiff(subtypes9, c("Hypodiploid", "Normal"))
subtypes5 <- setdiff(
  subtypes9,
  c("Hypodiploid", "Normal", "Hyperdiploid", "Others")
)
others <- data[, Y$subtype == "Others"]
others_normal <- data[, Y$subtype %in% c("Others", "Normal")]

#### Remove batch effect genes

In [150]:
# Prediction (Batch genes) -------------------------------------------------------
## Batch genes
# Only D0 samples
pid_d0 <- rownames(metadata_df)[metadata_df$class_info == "D0"]
pid_telaml1 <- rownames(metadata_df)[metadata_df$subtype == "TEL-AML1"]
pid_remission <- rownames(metadata_df)[metadata_df$label == 0]

# Recursive intersect
pid_idx <- Reduce(intersect, list(pid_d0, pid_telaml1, pid_remission, colnames(data)))

d0_telaml1 <- data[,pid_idx]
d0_batch <- metadata_df[colnames(d0_telaml1), "batch_info"]
d0_telaml1_t <- t(d0_telaml1)

#' @param X matrix with samples as rows and features as columns
calcBatchANOVA <- function(X, batch, method = "welch") {
  .featureANOVA <- function(vec, d0_batch, method) {
    X <- data.frame(gene = vec,
                    batch = as.factor(d0_batch))
    
    if (method == "welch") return(oneway.test(gene~batch, X)$p.value)
    else if (method == "aov") return(unname(unlist(summary(aov(gene~batch, data = X)))[9]))
    else if (method == "kruskal") return(kruskal.test(gene~batch, X)$p.value)
    else stop("option not available for argument: method")
  }
  
  pvalue <- sapply(data.frame(X), .featureANOVA, batch, method)
  names(pvalue) <- substring(names(pvalue), 2)
  n_nan <- sum(sapply(pvalue, is.na))
  print(c("No. of NaNs =", n_nan))
  return(pvalue)
}

aov_pvalue <- calcBatchANOVA(d0_telaml1_t, d0_batch, method = "aov")
# welch_pvalue <- calcBatchANOVA(d0_telaml1_t, d0_batch, method = "welch")
# kruskal_pvalue <- calcBatchANOVA(d0_telaml1_t, d0_batch, method = "kruskal")

# Selecting by pvalue threshold
batch_genes <- names(aov_pvalue)[aov_pvalue < 0.05 & !is.na(aov_pvalue)]
# welch_genes <- names(welch_pvalue)[welch_pvalue < 0.05 & !is.na(welch_pvalue)]
# kruskal_genes <- names(kruskal_pvalue)[kruskal_pvalue < 0.05 & !is.na(kruskal_pvalue)]
length(batch_genes)

[1] "No. of NaNs =" "16"           


[1] 2565

## Plot: UMAP (Vectors)

In [152]:
umap_others <- umap(t(others_normal))

In [134]:
#' Performs PCA and plots vectors. Assume: 3 normal samples
#' @param X dataframe with features x samples and containing
#' D0, D8 and 3 Normal patients (ordered correctly)
plot_vectors <- function(
  X, metadata_df, pca = T, cex = 3, main = NULL
) {
  # PCA
  if (pca) {
    pca_obj <- prcomp(t(X), scale = T)
    pca_df <- data.frame(pca_obj$x[,1:2])
    n <- nrow(pca_df)
    d0_pca <- pca_df[1:((n-3) / 2),]
    d8_pca <- pca_df[((n-1) / 2):(n-3),]
    
    stopifnot(all(substring(rownames(d0_pca), 1, 4) == substring(rownames(d8_pca), 1, 4)))
    
    subtype_pca <- cbind(d0_pca, d8_pca)
    colnames(subtype_pca) <- c("start_x", "start_y", "end_x", "end_y")
    norm_pca <- pca_df[(n-2):n,]
    
    # Obtaining batch and class annotations
    label <- as.numeric(metadata_df[rownames(d0_pca), "label"]) + 1
    batch <- as.factor(metadata_df[rownames(d0_pca), "batch_info"])
    print(label)
    
    # Axis labels
    eigenvalues <- (pca_obj$sdev)^2
    var_pc <- eigenvalues[1:4]/sum(eigenvalues)
    pc_labels <- sprintf("PC%d (%.2f%%)", 1:4, var_pc*100)
  } else {
    print("No PCA performed!")
    pca_df <- data.frame(X)
  }
  
  scatter_pca <- ggplot(data = subtype_pca) +
    geom_point(aes(x = start_x, y = start_y, colour = batch), shape = 15,
               size = cex, show.legend = T) +
    geom_point(aes(x = end_x, y = end_y, colour = batch), shape = 16,
               size = cex, show.legend = F) +
    geom_segment(aes(x = start_x, y = start_y, xend = end_x, yend = end_y),
                 arrow = arrow(length = unit(0.3, "cm")),
                 alpha = 0.5, colour = label) +
#     geom_text(aes(x = PC1_A, y = PC2_A,
#                   label = rownames(subtype_pca)),
#               position = position_nudge(x = 4, y = 2), size = 2.5) +
    geom_point(data = norm_pca, aes(x = PC1, y = PC2),
               size = cex, shape = 17)
  
  if (pca) {
    scatter_pca <- scatter_pca +
      xlab(pc_labels[1]) + ylab(pc_labels[2])
  }
  
  if (!is.null(main)) {
    scatter_pca <- scatter_pca + labs(title = main)
  }
  
  return(scatter_pca)
}

In [129]:
umap_others1 <- cbind(
  x = umap_others$layout,
  Y[rownames(umap_others$layout), ]
)

umap_others2 <- umap_others1[umap_others1$subtype == "Others", ]
umap_normal <- umap_others1[umap_others1$subtype == "Normal", ]
umap_others3 <- umap_others2[seq_len(nrow(umap_others2) / 2), ]
others_d8 <- umap_others2[seq(nrow(umap_others2) / 2 + 1, nrow(umap_others2)), 1:2]
umap_others4 <- cbind(umap_others3, others_d8)
umap_others4 <- umap_others4[, c(1,2,7,8,3:6)]
colnames(umap_others4)[1:4] <- c("start_x", "start_y", "end_x", "end_y")
umap_others4$batch_info <- as.factor(umap_others4$batch_info)
umap_others4$label <- as.factor(umap_others4$label)
arrow_col <- ifelse(umap_others4$label == 0, "black", "red")

scatter_pca <- ggplot(data = umap_others4) +
  geom_point(aes(x = start_x, y = start_y, colour = batch_info),
             shape = 15, size = cex, show.legend = T) +
  geom_point(aes(x = end_x, y = end_y, colour = batch_info),
             shape = 16, size = cex, show.legend = F) +
  geom_segment(
    aes(x = start_x, y = start_y, xend = end_x, yend = end_y),
    arrow = arrow(length = unit(0.3, "cm")),
    alpha = 0.5,
    color = arrow_col
  ) +
  geom_point(data = umap_normal, aes(x = x.1, y = x.2),
             size = cex, shape = 17)

ggsave("~/Dropbox/temp/umap-others.pdf", scatter_pca)

Saving 6.67 x 6.67 in image


## Plot: Vectors (PCA)

In [139]:
pca_all <- plot_vectors(others, Y)
ggsave("~/Dropbox/temp/vectors-pca_all.pdf", pca_all)

 [1] 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 2 2 2 1 2 2
[39] 1 2 1 1 1 1 2 2 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 2 1 2 1 1 1 2 1 1
[77] 1 1 2 1 1 1 1 2 1


Saving 6.67 x 6.67 in image


In [138]:
others_fltr <- others[!(rownames(others) %in% batch_genes), ]
pca_wo_batch <- plot_vectors(others_fltr, Y)
ggsave("~/Dropbox/temp/vectors-pca_wo_batch.pdf")

 [1] 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 2 2 2 1 2 2
[39] 1 2 1 1 1 1 2 2 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 2 1 2 1 1 1 2 1 1
[77] 1 1 2 1 1 1 1 2 1


Saving 6.67 x 6.67 in image


## Plot: Heatmap

In [223]:
others_d0 <- others[Y[colnames(others), "class_info"] == "D0"]

In [224]:
# exclude batch effect genes
others_d0_wo_batch <- others_d0[!(rownames(others_d0) %in% batch_genes), ]

#' Select top n highly variable genes
#' @param X matrix or dataframe with rownames
select_hvg <- function(X, n) {
  if (is.null(rownames(X)))
    stop("X does not have rownames")
  
  row_var <- apply(X, 1, var)
  sorted_var <- sort(row_var, decreasing = TRUE)
  hvg <- names(sorted_var)[1:n]
  X[hvg, ]
}

others_d0_hvg300 <- select_hvg(others_d0_wo_batch, 300)

# hierarchical clustering
heatmap_annot <- Y[, c("batch_info", "label"), drop = F]
others_clust <- pheatmap(
  others_d0_hvg300, col = brewer.pal(9, "Blues"),
  display_numbers = F,
  legend = T, border_color = "black", scale = "none",
  cluster_method = "ward.D2", cluster_rows = T, cluster_cols = T,
  cutree_cols = 4,
  show_colnames = T, show_rownames = T,
  annotation_col = heatmap_annot,
  fontsize = 2,
#   cellwidth = 10, cellheight = 1
  filename = "~/Dropbox/temp/heatmap-300hvg.pdf"
)

In [225]:
# retrieve column names of pheatmap
pid_ord <- others_clust$tree_col$labels[others_clust$tree_col$order]
pid_clusters <- cutree(others_clust$tree_col, k = 4)
print(pid_ord)
pid_clusters[pid_clusters == 3]

 [1] "P097_D0" "P158_D0" "P220_D0" "P204_D0" "P163_D0" "P213_D0" "P119_D0"
 [8] "P171_D0" "P123_D0" "P132_D0" "P065_D0" "P193_D0" "P194_D0" "P068_D0"
[15] "P214_D0" "P212_D0" "P060_D0" "P108_D0" "P086_D0" "P218_D0" "P078_D0"
[22] "P128_D0" "P071_D0" "P201_D0" "P130_D0" "P126_D0" "P150_D0" "P093_D0"
[29] "P089_D0" "P219_D0" "P125_D0" "P141_D0" "P062_D0" "P104_D0" "P080_D0"
[36] "P084_D0" "P209_D0" "P066_D0" "P067_D0" "P072_D0" "P064_D0" "P149_D0"
[43] "P181_D0" "P095_D0" "P210_D0" "P216_D0" "P101_D0" "P179_D0" "P200_D0"
[50] "P075_D0" "P202_D0" "P197_D0" "P199_D0" "P206_D0" "P217_D0" "P192_D0"
[57] "P205_D0" "P083_D0" "P063_D0" "P215_D0" "P087_D0" "P143_D0" "P174_D0"
[64] "P173_D0" "P177_D0" "P082_D0" "P059_D0" "P142_D0" "P187_D0" "P191_D0"
[71] "P092_D0" "P105_D0" "P076_D0" "P090_D0" "P111_D0" "P077_D0" "P073_D0"
[78] "P203_D0" "P081_D0" "P085_D0" "P061_D0" "P091_D0" "P070_D0" "P144_D0"
[85] "P180_D0"


P062_D0 P064_D0 P066_D0 P067_D0 P072_D0 P075_D0 P080_D0 P084_D0 P089_D0 P093_D0 
      3       3       3       3       3       3       3       3       3       3 
P095_D0 P101_D0 P104_D0 P125_D0 P126_D0 P130_D0 P141_D0 P149_D0 P150_D0 P179_D0 
      3       3       3       3       3       3       3       3       3       3 
P181_D0 P192_D0 P197_D0 P199_D0 P200_D0 P202_D0 P205_D0 P206_D0 P209_D0 P210_D0 
      3       3       3       3       3       3       3       3       3       3 
P216_D0 P217_D0 P219_D0 
      3       3       3

#### Feature selection might lead to overfitting
- Selecting highly D.E. features in remission samples
- If the D.E. features in relapse samples are independent, then highly unlikely that shift in relapse > remission

#### Heterogeneity
- Heterogeneity in the starting and ending point?
- Heterogeneity in response

#### Easy solutions
- No feature selection. Plot angle of optimal v.s. own response
- Use optimal as vector to project on

#### Assumption of states
- Assuming that it has to progress towards an end state
- Maybe it has to go to an intermediate state first
- If the state is the proportion of cells, then the above does not hold. i.e. Should progress towards normal patients
- Methotrexate: Antimetabolite
- Prednisolone: Steroid
- Vincristine: Inhibit cell division
- Dexamethasone: Counteract side effects
- Daunorubicin: Anthracycline (interfere with DNA metabolism and RNA production)

- Initial hypothesis:
    - Higher proportion of cancer cells to lower proportion
- New hypothesis:
    - Leukemia state: 100% blasts
        - Restricted to earlier states in developmental trajectory
        - Slight perturbation
    - Normal state: 100% normal WBCs
        - Full range in developmental traj
    - Response state: x% normal, (1-x)% blasts
        - All WBCs at later stage can be attributed to normal cells?
        - Assume that normal cells have the same distribution along developmental traj
        - Work out no. of normal cells from the no. of later stage cells
        - Total - later stage - normal early = No. of blasts
- Extension to bulk data
   - Find genes that are indicative of late stage normal cells?
   - Use ratios of gene expression to find out number of normal cells?
   - Gene expression that exceed that number will represent the blasts??

#### Batch effects
- Presence of batch effects?
- Delete the batch effects genes

#### Filtering and Aggregation
- Filtering out batch effect genes
- Aggregating by pathway info